# Installing Dependencies

In [ ]:

!pip install praw
!pip install transformers
!pip install rouge-score
!pip install bert-score
!pip install sentencepiece
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6792adf18daafc75c687f8188447bdad532a52dba30073af740d196edc5bcde5
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install --upgrade asyncpraw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 14.3 MB/s eta 0:00:00


In [ ]:
pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.2 MB/s eta 0:00:00


# Reddit Summarizer

Reddit summarizer that prints the summary of 10 posts of a given susbreddit

In [9]:
import asyncpraw
import asyncio
from transformers import pipeline
from transformers import BartTokenizer

# Setting up Reddit instance
async def setup_reddit():
    reddit = asyncpraw.Reddit(client_id = "k_-waUAoYeDwXPOXSw4RoA" ,
                        client_secret = "Ggq-0JiNR8GnzT9eQsfhiW80KrD6nA",
                        user_agent = "personal project"
                        )
    return reddit

# Fetching posts from a subreddit
async def fetch_posts(reddit, subreddit_name, limit=10):
    subreddit = await reddit.subreddit(subreddit_name)
    posts = subreddit.new(limit=limit)
    return posts

async def fetch_comments(post):
    await post.load()
    comments = post.comments.list()

    while True:
        more_comments = [comment for comment in comments if isinstance(comment, asyncpraw.models.MoreComments)]
        if not more_comments:
            break
        for more_comment in more_comments:
            await more_comment.replace_more(limit=32)
            comments = post.comments.list()

    return comments


def preprocess_text(comments):
    text = ""
    for comment in comments:
        if isinstance(comment, asyncpraw.models.Comment):
            text += comment.body + " "
            if len(text.split()) > 1024:
                break
    return text.strip()



def summarize_text(text):
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", tokenizer=tokenizer)

    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt")

    summary_ids = summarizer.model.generate(
        inputs.input_ids,
        max_length=130,
        min_length=30,
        num_beams=4,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [10]:
async def main():
    reddit = await setup_reddit()
    posts = await fetch_posts(reddit, "anime")

    async for post in posts:
        print(f"Title: {post.title}")
        comments = await fetch_comments(post)

        if comments:
            text = preprocess_text(comments)

            if text:
                summary = summarize_text(text)
                print(f"Summary: {summary}")
            else:
                print("No comments found.")
        else:
            print("No comments found.")

        print("\n")

    await reddit.close()


await main()

Title: Show by Rock!! 10th Anniversary Rewatch - Episode 9 Discussion


Device set to use cpu


Summary: First-Timer by Rock!! is the first episode of the new season of Plasmagica. First-Rocker: "The attacker wouldn't be coming back for round two if something hadn't changed"


Title: How many animes should i be watching at maximum at once?


Device set to use cpu


Summary: As many as you can handle and feel comfortable watching. This is entirely based on you, we can't answer it for you. For **source of fanarts**, try [SauceNAO](https://saucenao.com/) For **where to watch anime**, see [our list of streaming sites] or [search on Livechart.me]


Title: So, How Much Money Do Anime Studios Actually Make? - Official Gazette Numbers


...

Summary: The users of this subreddit came up with [a chart of anime recommendations for newcomers and veterans](https://i.redd.it/0w1v880jtq661.png) and [an awesome longer flowchart] Maybe you can find something there that you'll like.


Title: The Unaware Atelier Meister Has My Favourite Moment of the Season So Far


Device set to use cpu


Summary: It's only been four episodes, but I'm actually appreciating the show's approach to a character that can warp an entire nation's economy without breaking a sweat. Multiple factions keeping an eye on him, close friends trying to keep him safe while hopefully pushing him toward self-awareness, it's giving me hope that he'll experience character development.


Title: What Have You Watched This Past Week That is NOT a Currently Airing Show? [April 20th, 2025]


Device set to use cpu


Summary: Me last week: Tune in next time for most likely me starting YGO Duel Monsters properly for the first time, so I will no longer be a fake YGO fan. Also, the rest of Eureka Seven & more of the other shows.




# Reddit Summarizer

The version of reddit summarizer to print the summary of a given post using a url

Evaluation metrices like rouge score and bert score couldn't be used for this

*   Evaluation metrices like rouge score and bert score couldn't be used for this project because there are no reference summary present for every post to compare the output to.
*   So, evaluation metrices like semantic similarity, extractive analysis, and model readebility is evaluated.





In [6]:
import nest_asyncio
from transformers import BartTokenizer, pipeline
from rouge_score import rouge_scorer
from bert_score import BERTScorer
from sentence_transformers import SentenceTransformer, util
from difflib import SequenceMatcher
import textstat


async def setup_reddit():
    return asyncpraw.Reddit(
        client_id="k_-waUAoYeDwXPOXSw4RoA",
        client_secret="Ggq-0JiNR8GnzT9eQsfhiW80KrD6nA",
        user_agent="personal_project"
    )

def preprocess_text(title, selftext, comments):
    text = f"{title}\n{selftext}\n" + "\n".join(
        c.body for c in comments if isinstance(c, asyncpraw.models.Comment)
    )
    return text.strip()

def summarize_text(text):
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
    summarizer = pipeline("summarization",
                         model="facebook/bart-large-cnn",
                         tokenizer=tokenizer)

    inputs = tokenizer(
        text,
        max_length=1024,
        truncation=True,
        return_tensors="pt"
    )

    summary_ids = summarizer.model.generate(
        inputs.input_ids,
        max_length=130,
        min_length=30,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def semantic_evaluation(summary, original_text):
    model = SentenceTransformer('all-mpnet-base-v2')

    # Generate embeddings
    summary_embedding = model.encode(summary, convert_to_tensor=True)
    original_embedding = model.encode(original_text, convert_to_tensor=True)

    # Calculate cosine similarity
    similarity = util.pytorch_cos_sim(summary_embedding, original_embedding).item()
    return similarity

def extractive_fraction(summary, original_text):
    matcher = SequenceMatcher(None, summary.lower(), original_text.lower())
    matching_blocks = matcher.get_matching_blocks()

    total_matching = sum(block.size for block in matching_blocks)
    return total_matching / len(summary)

def readability_metrics(summary):
    return {
        "flesch_reading_ease": textstat.flesch_reading_ease(summary),
        "smog_index": textstat.smog_index(summary),
        "coleman_liau": textstat.coleman_liau_index(summary)
    }

def comprehensive_evaluation(summary, original_text):
    print("\n=== Automated Evaluation ===")

    sim_score = semantic_evaluation(summary, original_text)
    print(f"Semantic Similarity: {sim_score:.4f}")

    extr_score = extractive_fraction(summary, original_text)
    print(f"Extractive Fragment Score: {extr_score:.4f}")

    readability = readability_metrics(summary)
    print("\nReadability:")
    for metric, value in readability.items():
        print(f"- {metric}: {value:.2f}")

async def process_post(url):
    reddit = await setup_reddit()
    try:
        submission = await reddit.submission(url=url)
        await submission.load()

        # Get comments with error handling
        try:
            await submission.comments.replace_more(limit=0)
            comments = submission.comments.list()[:50]  # Top 50 comments
        except Exception as e:
            print(f"Error loading comments: {str(e)}")
            comments = []

        # Build text content
        text = preprocess_text(submission.title, submission.selftext, comments)

        if not text.strip():
            return None, "No content to summarize"

        summary = summarize_text(text)
        return summary, text

    finally:
        await reddit.close()

async def main():
    url = "https://www.reddit.com/r/india/comments/1jqe90g/i_am_a_trade_professional_in_the_government_of/"

    # Get results
    summary, original_text = await process_post(url)

    if summary:
        print("\nGenerated Summary:")
        print("-"*40)
        print(summary)

        comprehensive_evaluation(summary, original_text)

        print("\nOriginal Text Preview:")
        print("-"*40)
        print(original_text[:500] + "...")  # Show first 500 characters
    else:
        print("Failed to generate summary")

if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError:
        nest_asyncio.apply()
        asyncio.run(main())


Device set to use cpu



Generated Summary:
----------------------------------------
I am a trade professional in the government of India. Here's what you need to know about how US tariffs will affect you. US is the top location for remittances to India (NRIs sending money back to families in India), this will see a decline.

=== Automated Evaluation ===


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Semantic Similarity: 0.7802
Extractive Fragment Score: 1.0000

Readability:
- flesch_reading_ease: 64.71
- smog_index: 12.50
- coleman_liau: 6.60

Original Text Preview:
----------------------------------------
I am a trade professional in the government of India. Here's what you need to know about how US tariffs will affect you. AMA
[deleted]
Thank you for sharing your insights. Post like this are why I love Reddit.
What happens when the prices sky rockets for US consumers suddenly?
- Reduced spending
- Reduced demand
- Possible recession



What happens when the US market goes into recession?
- Services (like IT) which haven’t been touched yet, will be affected (due to lack of demand)
- Can lead the ...


<ipython-input-6-16834ef8fca6>:139: RuntimeWarning: coroutine 'main' was never awaited
  asyncio.run(main())
